In [1]:
import numpy as np

from src import PricingMethod, UnderlyingModel, Derivative

In [16]:
n_time = 10
n_mc = 3

dic_param_bs = {
    'r' : 0.01,
    'q' : [0.02, 0.1],
    'sigma' : np.array([[0.2, 0.4]]),
    'underlying_price' : [100,80],
    'rho' : np.array([[1.0, 0.5], [0.5, 1.0]]),
    'n_underlying' : 2
}

dic_param_call = {
    'K' : [100,90],
    'valuation_time' : 0,
    'end_time' : 1
}

european_call = Derivative.EuropeanCall(dic_param_call)
bs_model = UnderlyingModel.BlackScholes(dic_param_bs)
pricer_mc = PricingMethod.PricingMonteCarlo(n_time,n_mc)

ValueError: All elements in 'sigma' should be of type 'int' or 'float'.

In [18]:
dic_param_bs['sigma'].shape

(1, 2)

In [21]:
np.matmul(dic_param_bs['sigma'].T, dic_param_bs['sigma'])

array([[0.04, 0.08],
       [0.08, 0.16]])

In [6]:
print(bs_model)

Black Scholes model
r: 0.01
q: [0.02, 0.1]
sigma: [0.2 0.4]
underlying_price: [100, 80]
rho: [[1.  0.5]
 [0.5 1. ]]
n_underlying: 2



In [4]:
pricer_mc.generate_paths(european_call, bs_model)

[[1.        0.       ]
 [0.5       0.8660254]]


TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [11]:
l = [4,2,1,3,4,2]

In [12]:
l[4:6]

[4, 2]